# LVGLASSO comparison
This notebook shows a comparison between LGL and LTGL with respect to LVGLASSO, the implementation of the original model (for a single time point) presented here: https://ieeexplore.ieee.org/abstract/document/5707106

We compare with the code of the authors of https://arxiv.org/pdf/1206.1275, that reimplemented the method using ADMM, a minimisation procedure which we adopted as well.

In [ ]:
%matplotlib inline
import numpy as np
from sklearn.covariance import empirical_covariance

from regain import datasets
from regain.covariance import latent_graphical_lasso_, latent_time_graphical_lasso_

# config
np.random.seed(0)
n_samples = 100
n_dim_obs = 10
n_dim_lat = 2
T = 8
tau = 0.1
alpha = 0.1

dataset = datasets.make_dataset(
    n_samples=n_samples, n_dim_lat=n_dim_lat, update_ell='fixed',
    update_theta='l2', normalize_starting_matrices=True, n_dim_obs=n_dim_obs,
    T=T)

## 1. Single timestamp
Check if, with only one timestamp, the method behave the same as Chandrasekaran/Ma.

In [ ]:
emp_cov = empirical_covariance(dataset.data[0], assume_centered=False)

### 1.1 Do we behave as ourselves with the same functional as Ma?
The following is the latent time graphical model inference with only one covariance matrix.

In [ ]:
results_time = latent_time_graphical_lasso_.latent_time_graphical_lasso(
    emp_cov[None, ...], alpha=alpha, tau=tau, tol=1e-5, rtol=1e-5,
    rho=1. / emp_cov.shape[0], verbose=0, max_iter=500)

We compare it with the code for the latent graphical model inference (without time). <br>
Since there is only one covariance matrix, we expect to obtain the same results.

In [ ]:
results_static = latent_graphical_lasso_.latent_graphical_lasso(
    emp_cov, alpha=alpha, tau=tau, tol=1e-5, rtol=1e-5,
    rho=1. / emp_cov.shape[0], verbose=0, max_iter=500)

In [ ]:
assert np.all(
    [np.allclose(x, y) for x, y in zip(results_static, results_time)])
assert np.linalg.matrix_rank(results_static[1]) == np.linalg.matrix_rank(
    results_time[1][0])

### Optional (requires Matlab/Octave installed)
Now we check if the result is the same as the LVGLASSO Matlab algorithm. To do that, we implemented a simple wrapper which rely on `matlab.engine` or `oct2py`, to run Matlab code directly from Python. For `matlab.engine`, It requires Matlab 2016 or higher installed.

In [ ]:
from regain.wrapper import lvglasso
from sklearn.datasets.base import Bunch

result = lvglasso(emp_cov, alpha, tau, 1. / emp_cov.shape[0])
ma_output = Bunch(**result)

assert np.all(
    [
        np.allclose(x, y, atol=1e-3)
        for x, y in zip(results_static[:2], (ma_output.S, ma_output.L))
    ])
assert np.linalg.matrix_rank(ma_output.L) == np.linalg.matrix_rank(
    results_time[1][0])

## 2. Time-varying vs separate for each time
This is to justify the choice of the additional penalties which constrain subsequent matrices in time to behave similarly.

In [ ]:
# config
n_samples = 100
n_dim_obs = 10
n_dim_lat = 2
T = 8

np.random.seed(0)

dataset = datasets.make_dataset(
    n_samples=n_samples, n_dim_lat=n_dim_lat, update_ell='fixed',
    update_theta='l2', normalize_starting_matrices=True, n_dim_obs=n_dim_obs,
    T=T)

X, y = dataset.X, dataset.y

First, we check again if the results are the same with beta and eta is 0.

In [ ]:
emp_cov = np.array(
    [
        empirical_covariance(data, assume_centered=False)
        for data in dataset.data
    ])
emp_list = np.array(emp_cov).transpose(1, 2, 0)

In [ ]:
results_time = latent_time_graphical_lasso_.latent_time_graphical_lasso(
    emp_cov, alpha=alpha, tau=tau, tol=1e-5, rtol=1e-5,
    rho=1. / emp_cov.shape[0], beta=0, eta=0, verbose=0, max_iter=500)

results_static = [
    latent_graphical_lasso_.latent_graphical_lasso(
        x, alpha=alpha, tau=tau, tol=1e-5, rtol=1e-5,
        rho=1. / emp_cov.shape[0], verbose=0, max_iter=500) for x in emp_cov
]

In [ ]:
assert np.all(
    [
        np.allclose(results_static[i][0], results_time[0][i], atol=1e-2)
        for i in range(T)
    ])
assert np.all(
    [
        np.linalg.matrix_rank(results_static[i][1]) == np.linalg.matrix_rank(
            results_time[1][i]) for i in range(T)
    ])

Again, optional: it requires Matlab or Octave installed.

In [ ]:
result = lvglasso(emp_list, alpha, tau, rho=1. / emp_list[0].shape[0])
ma_output = Bunch(**result)

# note time is the last dim, as emp_list
ma_output.R = np.array(ma_output.R.T)
ma_output.S = np.array(ma_output.S.T)
ma_output.L = np.array(ma_output.L.T)

assert np.allclose(results_time[0], ma_output.R + ma_output.L, atol=1e-2)

In [ ]:
np.mean(results_time[0] - (ma_output.R + ma_output.L))

In [ ]:
L = np.array(ma_output.L)
LL = results_time[1]

ranks_ma = [np.linalg.matrix_rank(l) for l in L]
ranks_ours = [np.linalg.matrix_rank(l) for l in LL]
assert np.all(
    [
        np.linalg.matrix_rank(l) == np.linalg.matrix_rank(ll)
        for l, ll in zip(L, LL)
    ])

# Penalty contribution

We now checked that in the limit case of one time and in the case in which we do not consider the penalties that involve time we perform equivalentely. Now, with CV on the parameters on synhtetic data generated with norm2 we want to see if our method performs better than LVGLASSO applied on different time stamps. 

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from regain import utils
from regain.covariance import LatentTimeGraphicalLasso

ltgl = GridSearchCV(
    LatentTimeGraphicalLasso(),
    dict(
        tau=np.logspace(-2, np.log(.5), 10),
        alpha=np.logspace(-2, np.log(.5), 10)), cv=StratifiedShuffleSplit(10),
    return_train_score=True).fit(X, y)

In [ ]:
emp_list = list(ltgl.best_estimator_.covariance_)
alpha = ltgl.best_params_['alpha']
tau = ltgl.best_params_['tau']

In [ ]:
precs = []
lats = []
covss = []
for data in dataset.data:
    lgl = LatentGraphicalLasso(tau=tau, alpha=alpha,
                               assume_centered=False).fit(X, y)
    precs.append(lgl.precision_)
    lats.append(lgl.latent_)
    covss.append(lgl.covariance_)

### Error obtained w.r.t. the observed data

In [ ]:
from regain.utils import error_norm_time
print(
    "Error norm time with observed precision: \n LTGL: {:.3f}\n LVGLASSO: {:.3f}"
    .format(
        error_norm_time(
            ltgl.best_estimator_.precision_ - ltgl.best_estimator_.latent_,
            dataset.thetas_observed),
        error_norm_time(
            np.array(precs) - np.array(lats), dataset.thetas_observed)))

### Error obtained w.r.t. the real data

In [ ]:
print(
    "Error norm time of LTGL: \n Precision: {:.3f}\n Latent: {:.3f}\n".format(
        error_norm_time(ltgl.best_estimator_.precision_, dataset.thetas),
        error_norm_time(ltgl.best_estimator_.latent_, dataset.ells)))

print(
    "Error norm time of LVGLASSO: \n Precision: {:.3f}\n Latent: {:.3f}".
    format(
        error_norm_time(np.array(precs), dataset.thetas),
        error_norm_time(np.array(lats), dataset.ells)))

### Error in ranks

In [ ]:
# ltgl.best_estimator_.set_params(beta=1e12,eta=1e12).fit(data_grid)
print(
    "Rank latent matrices LTGL: {}".format(
        [np.linalg.matrix_rank(i) for i in ltgl.best_estimator_.latent_]))
print(
    "Rank latent matrices LVGLASSO: {}".format(
        [np.linalg.matrix_rank(i) for i in lats]))
print(
    "Rank true latent matrices: {}".format(
        np.linalg.matrix_rank(dataset.ells)))

### Error in structure

In [ ]:
from regain.utils import structure_error

print(
    "f1 score LTGL: {:.2f}".format(
        structure_error(
            dataset.thetas, ltgl.best_estimator_.precision_, thresholding=1,
            eps=1e-2)['f1']))
print(
    "f1 score LVGLASSO: {:.2f}".format(
        structure_error(
            dataset.thetas, np.array(precs), thresholding=1, eps=1e-2)['f1']))

# Scalability

In [ ]:
from tqdm import tqdm

n_samples = 100
# n_dim_obs = 10
n_dim_lat = 2
T = 10
np.random.seed(0)
import time
results = {}

for n_dim_obs in tqdm(np.logspace(3, 6, 10)):
    dataset = datasets.make_dataset(
        n_samples=n_samples, n_dim_lat=n_dim_lat, update_ell='fixed',
        update_theta='l2', normalize_starting_matrices=True,
        n_dim_obs=n_dim_obs, T=T)

    X, y = dataset.X, dataset.y

    emp_cov = np.array(
        [
            empirical_covariance(data, assume_centered=False)
            for data in dataset.data
        ])
    emp_list = np.array(emp_cov).transpose(1, 2, 0)

    res, elapsed_time = [], []
    for x in emp_cov:
        tic = time.time()
        results_static = latent_graphical_lasso_.latent_graphical_lasso(
            x, alpha=alpha, tau=tau, tol=1e-5, rtol=1e-5,
            rho=1. / emp_cov.shape[0], verbose=0, max_iter=500)
        tac = time.time() - tic

        res.append(results_static)
        elapsed_time.append(tac)

    results[('lgl', n_dim_obs)] = dict(res=res, elapsed_time=elapsed_time)

    res, elapsed_time = [], []
    for x in emp_cov:
        tic = time.time()
        result = lvglasso(x, alpha, tau, 1. / x.shape[0])
        ma_output = Bunch(**result)

        ma_output.R = np.array(ma_output.R)
        ma_output.S = np.array(ma_output.S)
        ma_output.L = np.array(ma_output.L)
        tac = ma_output.elapsed_time  #time.time() - tic

        res.append(ma_output)
        elapsed_time.append(tac)

    results[('lvglasso',
             n_dim_obs)] = dict(res=ma_output, elapsed_time=elapsed_time)
#     results[('lvglasso', n_dim_obs)] = ma_output

# assert np.allclose(results_time[0], ma_output.R + ma_output.L, atol=1e-3)

#     L = np.array(ma_output.L)
#     LL = results_time[1]

#     ranks_ma = [np.linalg.matrix_rank(l)for l in L]
#     ranks_ours = [np.linalg.matrix_rank(l)for l in LL]
# assert np.all([np.linalg.matrix_rank(l) == np.linalg.matrix_rank(ll) for l, ll in zip(L, LL)])

In [ ]:
from regain.utils import flatten
df = pd.DataFrame(
    flatten(
        [
            [
                (k[0], int(k[1] * k[1] - 1) // 2 * T, x)
                for x in results[k]['elapsed_time']
            ] for k in results
        ]), columns=['method', 'n_unknowns', 'time'])

import seaborn as sns
from matplotlib import pyplot as plt
f, ax = plt.subplots()
g = sns.pointplot(data=df, hue='method', x='n_unknowns', y='time')
g.set_yscale("log");